In [1]:
from visualizer import calibration
#TODO: import from waveorder
import visual 

from PyQt5 import QtCore

from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
import tifffile

from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io
import zarr
import cupy as cp 
# import imagej

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [13]:
# main_folder = r'F:\DATA\20220507_M25_Zw495_9'
# 
# main_folder =r'H:\Test\20220507_M25_ZW495_Egg_Timelapse'  #DIED 
# main_folder =r'F:\DATA\20220507_M25_ZW495_19'   # 5 worm in all FOV and some movement
# main_folder =r'H:\Test\20220507_M25_ZW495_18'  #3 starts in FOV but then moves out
# main_folder =r'H:\Test\20220507_M25_ZW495_17'  # 3 some movement in 3D and some beads are on FOV
# main_folder =r'H:\Test\20220507_M25_ZW495_16'  #3 thicc one with barely any movement
# main_folder =r'H:\Test\20220507_M25_ZW495_15'  #3 just the tail
# main_folder =r'F:\DATA\20220507_M25_ZW495_14' #5 2 worms in FOV.
# main_folder =r'H:\Test\20220507_M25_ZW495_13' #3 movement at the edge
# main_folder =r'H:\Test\20220507_M25_ZW495_12'  #3 not focus on center planes
# main_folder =r'H:\Test\20220507_M25_ZW495_11' #3 not much movement and out of focus on central planes
# main_folder =r'H:\Test\20220507_M25_ZW495_10'  #4 worth taking a look since they move in 3D
# main_folder =r'F:\DATA\20220507_M25_ZW495_9' #5 moves in FOV. Curling up and then shooting out
# main_folder =r'H:\Test\20220507_M25_ZW495_8'  #3 movement out of FOV
# main_folder =r'H:\Test\20220507_M25_ZW495_7'  # 5 lots of worms in FOV coming in and out.
# main_folder =r'F:\DATA\20220507_M25_ZW495_6'  # 5 lots of twitching in FOV
# main_folder =r'H:\Test\20220507_M25_ZW495_5'  #3 out FOV movement
# main_folder =r'H:\Test\20220507_M25_ZW495_4'  #5 amazing movement in 3D
# main_folder =r'F:\DATA\20220507_M25_ZW495_3'  #5 movement in 3D
# main_folder =r'F:\DATA\20220507_M25_ZW495_2'  #5 fast dynamics on second half
# main_folder =r'F:\DATA\20220507_M25_ZW495_1'  #5 fast dynamics of big worm

## Datasets from 05/08
# main_folder = r'H:\ZW495\20220508_M25_zw495_1'
# main_folder = H:\ZW495\20220508_M25_zw495_2
# main_folder = H:\ZW495\20220508_M25_zw495_3
# main_folder = r'H:\ZW495\20220508_M25_zw495_4'  #3 dead floating
# main_folder = r'H:\ZW495\20220508_M25_zw495_6'  #3 moves at the lower side of the FOV
# main_folder = r'H:\ZW495\20220508_M25_zw495_8' # 1 egg sample fluorescence
# main_folder = H:\ZW495\20220508_M25_zw495_9_WL
# main_folder = H:\ZW495\20220508_M25_zw495_10_WL
# main_folder = r'H:\ZW495\20220508_M25_zw495_11'
# main_folder = r'H:\ZW495\20220508_M25_zw495_12' #4 worms in FOV making it hard to analyze but good for dynamics
# main_folder = r'H:\ZW495\20220508_M25_zw495_13' #4 some movement aroudn the FOV. not for analysis
# main_folder = r'H:\ZW495\20220508_M25_zw495_14'  #4 THIS WORM SHOWS THATA MAYBE THERE IS NO CALCIUM ACTIVITY?
# main_folder = r'H:\ZW495\20220508_M25_zw495_15' #5 worth analyzing. does from around the FOV...
# main_folder = r'H:\ZW495\20220508_M25_zw495_17'  #2 no swimming in FOV
# main_folder = r'H:\ZW495\20220508_M25_zw495_18'  #3 Not much movement single worm on the FOV.
# main_folder = r'H:\ZW495\20220508_M25_zw495_19'  #4 Single worm with some movement. For some reason there is a huge shift in z halfway through
# main_folder = r'H:\ZW495\20220508_M25_zw495_20' #3 movement in 3D. oscillates int he z dimension
# main_folder = r'H:\ZW495\20220508_M25_zw495_22' #4 some movement of single worm. not swimming behavior... 
# main_folder = r'H:\ZW495\20220508_M25_zw495_23'  #2 very still worm. not 
# main_folder = r'H:\ZW495\20220508_M25_zw495_24' #2 Z KEEPS SINKING?!
# main_folder = r'H:\ZW495\20220508_M25_zw495_25' #1 unusabel
# main_folder = r'H:\ZW495\20220508_M25_zw495_26' #5 worth using the algorithm to analyze
# main_folder = H:\ZW495\20220508_M25_zw495_27
# main_folder = H:\ZW495\20220508_M25_zw495_28
# main_folder = r'H:\ZW495\20220508_M25_zw495_29'



# main_folder =r'H:\Data\KellerExplant_ian\20220508_M25_Iansample'  #5 fast dynamics of big worm

# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'  #3 first frames are slow
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'   #5 fast dynamics of big worm
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'   #5 fast dynamics of big worm
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'  #5 fast dynamics of big worm
# main_folder =r'H:\Data\OH15265\20220508_M25_oh15265_1'   #5 fast dynamics of big worm


# main_folder='F:\DATA\20220504_M25_celegansOH_late2'

### OH15265 WORMS
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_1' #2 Nothign in plane. Good for Background Subtractiong?

# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_2' #3 fast and dynamic but goes out of FOV a lot
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_3' #5 head of worm is not moving much. good for neuro
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_4' #5 great movement in 3D and across FOV
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_5' # *** Are we photobleaching?
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_7' #3 at the edge of FOV comes in and out of FOV
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_8' #5 3 worms in FOV. good for 3D neuro
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_9' #3 very bright worm laying in Z. some dynamics.
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_10' #5 starts with worm in FOV. Shoots out off FOV fast
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_11' #3 two worms. no moving..
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_12' #4 curled up. not much movement.
main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_13' #5 in FOV, some dynamic. good signal
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_14' #5 Very FAST dynamic inside FOV for most capture.
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_16' #5 curled up L2. Good for neural segmentation.not much movement
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_17' #5 some moving in center of FOV. makes a V shape
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_18' #5 amazing moving in 3D. worm is acros the FOV horizontally which works for all planes.
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_19' #4 curled up head of worm. good signal.
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_20' #4 good swimming. unfortunately goes to lower FOV and one cam crops that field.
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_21' #3 noisy
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_22' #5 L2 moving. Lots of dynamics in FOV. amazing signal
# main_folder =r'E:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_23' #5 L1 moving! amazing neurons.
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_24' #3 barely in FOV
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_25' #4 worm flat on z so hard to tell
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_27' #5 if we want tos how multiple curled up worms
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_28' #4 most worms leave the FOV
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_29' #5 lots of dynamics of single worm!!
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_30' #3 lots of worms and 3D dynamics not sure for analysis..
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_31' #5 adult worm. half body. nerons bright and big
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_32' #5 whole worm in FOV. some movement!

# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_33'   #3 good signal no movement.
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_34' # 3 dynamics of worm spazing in 3D
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_35' # 5 good 3D and movement
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_36' #5 great signal and movement in 3D
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_37'  #5 great 3D and whole worm in FOV
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_38' #3 multiple worms in FOV
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_39' #4 many worms in FOV. challenging for gracking?
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_40' #5 good for neural analysis . good signal
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_41' #5 good for neural analysis . good signal
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_42' #4 WF imaging of worms
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_43' #4 good for neural analysis . good signal
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_45' #5 fast 3D movements. good signal
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_46' #5 a little high on FOV but good signal
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_47' #5 half of the body is in. Worth doing analysis?
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_48' #3 exists the FOV
# main_folder =r'F:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_49' #3 very noisy. moves in 3D
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_50' #5 great to see all the neurons.
# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_51' #5 movement and then curl up



# main_folder =r'D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_40' #5 good for neural analysis . good signal



# main_folder =r'H:\Data\KellerExplant_ian\20220508_M25_kelleexplant_ian'   #5 fast dynamics of big worm



#Scope Parameters
# FOV = 50e-6
cam_px = 6.0e-6
totalmag = 15.75
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img

print(px_size_img)
print(zstep/cam_px)
print(z_scale)


center_planes = calibration.lazy_dask_stack(main_folder,num_cams=5,px_depth='uint16', height=608, width =608)
# center_planes = calibration.lazy_dask_stack(main_folder,num_cams=3,px_depth='uint8', height=608, width =608)

napari.view_image(center_planes, name='center_planes',scale=[z_scale,1,1],multiscale=False)

3.8095238095238096e-07
0.3333333333333333
5.25


c:\ProgramData\Anaconda3\envs\m25-py39\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 303.5, 303.5), zoom=2.0765624999999996, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 600.0, 1.0), (0.0, 26.25, 5.25), (0.0, 608.0, 1.0), (0.0, 608.0, 1.0)), current_step=(300, 2, 304, 304), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'center_planes' at 0x15dcca93070>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(poi

In [3]:
viewer= napari.Viewer()


c:\ProgramData\Anaconda3\envs\m25-py39\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [4]:
def start_imagej():
    import imagej
    global ij
    ij = imagej.init( mode='interactive')
    # path = r'C:\Users\yoshi\Documents\Fiji.app'
    # ij = imagej.init(path,mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)
# Align the files
ij_stack2  = ij.py.to_java(np.array(stack))
ij.ui().show('PSF', ij_stack2)

NameError: name 'ij' is not defined

### Loading center planes for evaluation

In [16]:
%%time
#Load Only center Camera to see if good FOV and signal

# center_planes = calibration.lazy_dask_stack(main_folder,num_cams=10,px_depth='uint8', height=608, width =808)

center_planes = calibration.lazy_dask_stack(main_folder,num_cams=5,px_depth='uint16', height=608, width =608)

CPU times: total: 641 ms
Wall time: 672 ms


In [17]:
napari.view_image(center_planes, name='center_planes',scale=[z_scale,1,1],multiscale=False)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 303.5, 303.5), zoom=0.8953125, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 450.0, 1.0), (0.0, 26.25, 5.25), (0.0, 608.0, 1.0), (0.0, 608.0, 1.0)), current_step=(225, 2, 304, 304), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'center_planes' at 0x2f41ebdeb80>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None,

### Loading Dataset

In [14]:
%%time

# stack = calibration.lazy_dask_stack(main_folder,num_cams=25,px_depth='uint8', height=600, width =960)

stack = calibration.lazy_dask_stack(main_folder,num_cams=25,px_depth='uint16', height=608, width =608)

CPU times: total: 4.16 s
Wall time: 4.36 s


### Align Datasets with Known Offsets


In [19]:
# Load offsets from desired folder 
processed_files = main_folder + '/'+'processed_files'
if not os.path.exists(processed_files):
    os.makedirs(processed_files)

main_folder_offset = r'D:\Data\MBL_ED_ANTONE\PSF\200nm\220508_380nmSteps_minusplus_reversed_1'
offsets_file_load = os.path.join(main_folder_offset,"offsets.csv")
with open(offsets_file_load) as file_name:
    offsets = np.loadtxt(file_name, delimiter=",")

In [15]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
        
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

(25, 3)
(600, 25, 608, 608)


In [17]:
#Convert numpy to dask?
da_stack_align = da.from_array(stack_aligned)
da_stack_align[:,12,:,:]= da_stack_align[:,12,:,:]*1.3

print(da_stack_align)

dask.array<setitem, shape=(600, 25, 608, 608), dtype=uint16, chunksize=(120, 25, 76, 76), chunktype=numpy.ndarray>


In [4]:
viewer= napari.Viewer()

c:\ProgramData\Anaconda3\envs\m25-py39\lib\site-packages\napari_tools_menu\__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [9]:
## View partial planes
da_stack_align_partial = da.from_array(stack_aligned[:,5:22,:,:])
viewer.add_image(da_stack_align_partial, name='dask_align',scale=[z_scale,1,1], multiscale=False)

<Image layer 'dask_align' at 0x15dde6c7af0>

In [87]:
viewer= napari.Viewer()

In [14]:
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1], multiscale=False)

<Image layer 'dask_align' at 0x226f62f11c0>

### SAVE TIFF ALIGNED

In [18]:
import zarr

filename = 'aligned_stack' + '.zarr'
filepath_save = os.path.join(processed_files,filename)
print(filepath_save)
zarr.save(filepath_save,stack_aligned)

D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_50/processed_files\aligned_stack.zarr


KeyboardInterrupt: 

In [20]:
import tifffile

filename = 'aligned_stack' + '.ome.tif'
filepath_save = os.path.join(processed_files,filename)
print(filepath_save)
tifffile.imwrite(filepath_save,stack_aligned)

D:\Data\MBL_ED_ANTONE\oh15265_2\20220509_M25_oh15265_13/processed_files\aligned_stack.ome.tif


In [18]:
# # Align the files
# ij_stack2  = ij.py.to_java(np.array(da_stack_align,dtype='uint16'))
# ij.ui().show('PSF', ij_stack2)

### CROP PSF TO ANALYZE


In [9]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)

KeyError: "'Shapes' is not in list"

In [13]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
# np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
print(t,c)
crop_img = da_stack_align[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

240 8


<Image layer 'crop' at 0x1f92f811b50>

### Image J Part

In [3]:
viewer= napari.Viewer()

In [3]:
def start_imagej():
    import imagej
    global ij
    # ij = imagej.init( mode='interactive')
    path = r'C:\Users\yoshi\Documents\Fiji.app'
    ij = imagej.init(path,mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)

In [7]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(stack))
ij.ui().show('PSF', ij_stack2)

In [15]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(da_stack_align))
ij.ui().show('PSF', ij_stack2)

In [16]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)
# anp2 = anp[:,:,:,np.newaxis]
# print(anp2.shape)
# anp2 = np.swapaxes(anp,1,4)
# anp2.shape

(60, 342, 413, 25)


In [17]:
anp2 = np.swapaxes(anp,1,3)
print(anp2.shape)
anp2_da = da.asarray(anp2)
viewer.add_image(anp2_da, name='crop',scale=[z_scale,1,1],multiscale=False)

(60, 25, 413, 342)


<Image layer 'crop' at 0x22682c33220>

#### Save TIFF FIles

In [21]:
import tifffile

filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)
print(filepath_save)
tifffile.imsave(filepath_save,anp2_da)

H:\Test\20220503_M25_moreCelegans10pm\crop_section.ome.tif


C:\Users\yoshi\AppData\Local\Temp\ipykernel_17792\34166168.py:6: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(filepath_save,anp2_da)


### Read TIFF FILE

In [5]:
viewer = napari.Viewer()

In [8]:
### For preexisting tiff saved datasets
filename = 'aligned_stack.ome.tif'
filepath_save = os.path.join(processed_files,filename)

In [5]:
filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)

In [9]:
crop_tiff = tifffile.imread(filepath_save)
print(crop_tiff.shape)
print(type(crop_tiff))
da_crop_tiff = da.from_array(crop_tiff)

viewer.add_image(crop_tiff, name='crop',scale=[z_scale,1,1],contrast_limits=(0,2**16-1))

(735, 25, 608, 608)
<class 'numpy.ndarray'>


<Image layer 'crop' at 0x231a0097d30>

In [9]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(crop_tiff))
ij.ui().show('PSF', ij_stack2)

In [ ]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)

In [8]:
from napari_animation import Animation
animation = Animation(viewer)
animation.animate('test.gif',canvas_only = False )

### ZARR IMPORT

In [12]:
main_folder = r'H:\Test\20220504_M25_celegansOH_late1'
file_name = main_folder + '/'+'deconvolved_worm' + '.zarr'
zarr_stack = zarr.load(file_name)
da_stack_deconvolved = da.from_array(zarr_stack)


In [14]:
viewer.add_image(da_stack_deconvolved, scale= [z_scale,1,1])

<Image layer 'da_stack_deconvolved [1]' at 0x1351614dbe0>

c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private

### Playing with napari-animation plugin


In [41]:
from napari_animation import AnimationWidget

animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

ModuleNotFoundError: No module named 'napari_animation'